In [ ]:
import pandas as pd
import numpy as np

In [ ]:
def extract_iv(data, y_col, bins=10):
    
    df_IV = pd.DataFrame()
    cols = data.columns
    x_col = cols[~cols.isin([y_col])]
    
    for i in x_col:
        if (data[i].dtype.kind in 'fbci') and (len(np.unique(data[i]))>10):
            quartx = pd.qcut(data[i], bins,  duplicates='drop')
            cart = pd.DataFrame({'range_x': quartx, 'y': data[y_col]})
        else:
            cart = pd.DataFrame({'range_x': data[i], 'y': data[y_col]})

        cart_sum = cart.groupby("range_x", as_index=False).agg({"y": ["count", "sum"]}) 
        cart_sum.columns = ['Cutoff', 'Total', 'Events']
        
        cart_sum['Percent Events'] = np.maximum(cart_sum['Events'], 0.5) / cart_sum['Events'].sum()

        cart_sum['Non-Events'] = cart_sum['Total'] - cart_sum['Events']
        cart_sum['Percent Non-Events'] = np.maximum(cart_sum['Non-Events'], 0.5) / cart_sum['Non-Events'].sum()

        cart_sum['Weight_Evidence'] = np.log(cart_sum['Percent Events']/cart_sum['Percent Non-Events'])
        cart_sum['Information_Value'] = cart_sum['Weight_Evidence'] * (cart_sum['Percent Events'] - cart_sum['Percent Non-Events'])
        cart_sum.insert(loc=0, column='Feature', value=i)
        temp =pd.DataFrame({"Feature" : [i], "Information_Value" : [cart_sum['Information_Value'].sum()]}, columns = ["Feature", "Information_Value"])
        df_IV=pd.concat([df_IV,temp], axis=0)

    return df_IV